# Mixtral of Experts
ARXIV, Mixtral.ai | Submitted on 8 Jan 2024


*Albert Q. Jiang et al.*

### Links:
- ARXIV publication of the paper - [link](https://arxiv.org/abs/2401.04088)
- PDF of the article - [link](https://arxiv.org/pdf/2401.04088.pdf)
- The provided code by the authors of the paper - [link 1](https://github.com/mistralai/mistral-src) and [link 2](https://github.com/mistralai/mistral-src/blob/main/mistral/model.py)
- Motivational video by the paper's authors (it was in the research paper for some reason) - [link](https://www.youtube.com/watch?v=dQw4w9WgXcQ)

### Notes:
- When mentioning section and/or equation, it is referred as the section and/or equation in the paper that we are summarizing.
    - Example: Equation (4) in section 2.2.1 expands the formula... 
    
- When referring to a section and/or equation that is in this work it will be explicitly noted as to avoid confusion.

### Quick Introduction
Mixtral 8x7B is a sparse MoE model that uses 13B active (47B in total) parameters that outperforms Llama 2 70B and GPT-3.5 on almost all LLM benchmarks that are used in this paper lagging behind just GPT4. The thing that makes this model special is the low computational cost to quality in accuracy ratio. Mixtral 8x7B can do this by implementing a MoE architecture as well as specific fine-tuning and optimization techniques. It differs from Mistral 7B by having 8 experts for each layer.

### Summary 

<img src="../media_rs/router.png" style="height:300px" />

#### What are MoE models?
MoE models are a relatively old concept (Jacobs et al., 1991; Jordan & Jacobs, 1994). The essential idea is to have $n$-number of neural nets and a gate (router) function which we both train as to:

1) Get the gate function to choose the right neural net for the given task;
2) Get the NNs to perform well on a specific task.

This approach has been implemented by Shazeer et al. (2017) who uses sparse MoE as a LM. His model consisted of 131072 experts! When given a token the router takes the 3 best performing models and uses them. The interesting observation was that different experts learn to do different things, like one only learns punctuation and a second only learns to use verbs (figure 8. of this study shows this)!

#### Mixtral 8x7B

In this context, Mixtral 8x7B is a MoE model with max input token size of 32k. The model is comprised of 8 experts each of which has 7B parametres (this is where the "8x7B" part comes from). On every layer 2 experts are called upon for every token input. Their results are then combined. The implementation of only 2 experts per token is one of the reason for its speed. The model boasts about high inference speeds based on the fact that it is a sparse MoE (SMoE) that uses only a fraction of the data (layer operations are represented as sparse matrix multiplications) plus the fact that different layers can be distributed to different GPUs.

The layers can be seen in the table below:

<img src="../media_rs/table_m.png" style="height:300px"/>

Mixtral-8x7b uses supervised fine-tuning and direct preference optimization, the latter of which is a reworked reward model used in RLHF proposed by [Rafael Rafailov el al.](https://arxiv.org/abs/2305.18290), reaching a score of 8.30 on MT-Bench.


#### How a MoE layer works

Basically the output of MoE layers for a given input $x$ is a weighted sum of the outputs of $n$-number of expert networks. The weights are given by the gating network (router) so that if we have $n$ expert networks, the output is given by: 

$$\sum_{i=0}^{n-1} G(x)_i \cdot E_i(x)$$

Where $G(x)_i$ is the gating network's output for the $i$-th expert and $E_i(x)$ is the output of the expert network. $G(x)_i$ is mainly implemented by taking the $\text{softmax}$ over Top-K logits (K is the number of experts per token):

$$G(x) \coloneqq \text{softmax}(\text{TopK}(x \cdot W_g))$$

In general execution an MoE layer is applied on every token. Also the [SwiGLU](https://abdulkaderhelwan.medium.com/swiglu-activation-function-77627e0b2b52) activation is being used for $E_i(x)$ and K is set to 2. All in all a single output for a token looks like: 
$$y = \sum_{i=0}^{n-1} \text{Softmax}(\text{Top2}(x \cdot W_g))_i \cdot \text{SwiGLU}_i(x)$$

#### Experimental results

The team at Mistral.ai test Mixtral and Llama on 5 benchmarks featuring commonsense reasoning, world knowledge, reading comprehension, math, code and popular aggregated results:

<img src="../media_rs/mixtral2.png" style="height:400px" />

As one can see the model exceeds or is on par with the other mentioned models, but it uses a fraction of the computational cost needed to run Llama 2 13 to 70B. We can see this visually in the next graph:

<img src="../media_rs/mixtral3.png" style="height:400px" />

Where we can see that Mistral 8x7B managers higher or same score compared to Llama70B but with a little less than 10x number of active parameters. 

Mixtral 8x7B also has 100% retrieval accuracy on the passkey retrieval task (section 3.2) and less bias score on BBQ (Bias Benchmark for QA) and BOLD (Bias in Open-Ended Language Generation
Dataset) than Llama 70B. When talking about bias it is nice to show that for different tasks (like philosophy or reasoning) the model doesn't show bias towards choosing always the same expert **except** for mathematics:

<img src="../media_rs/experts.png" style="height:500px" />

The different colors are the different domains in which the model works. As seen for mathematics model 2 is used more frequently which the authors attribute to the synthetic nature of the used dataset for training (The Pile validation dataset) as well as the limited access to all the languages (section 5.).

> Which I don't really get as an explanation. There is also a mention about the correlation of the hidden states in the first layers are more correlated to the input and output of the model. Another reason for why we need more XAI methods. 

## Code reproduction

Loading Mixtral 8x7b on my current hardware or on the cloud is impossible for me without some optimization methods like offloading and quantization of the model's weights. Some work has already gone into optimizing the model, mainly the research by Artyom Eliseev and Denis Mazur ([link](https://arxiv.org/pdf/2312.17238v1.pdf) to the paper). We can use the code provided by them to run Mixtral 8x7b Instruct with 12gb of VRAM, but even though I managed to load the model with its weights, I encountered plethora of issues with the output of the model.

I have used Mixtral 8x7b Instruct, because Mixtral 8x7b failed to load every single time because of OOM errors.

I have worked in 2 notebooks in kaggle. **Notebook 1** (Mixtral 8x7b) is an attempt to use Bits and Bytes for weights quantization into 4 and 8 bits. **Notebook 2** (Mixtral 8x7b v2.0) uses offloading and the library created by  Artyom Eliseev and Denis Mazur. Links:

- [Notebook 1](https://www.kaggle.com/code/momchilgeorgiev/mixtral-8x7b) (Mixtral 8x7b)
- [Notebook 2](https://www.kaggle.com/code/momchilgeorgiev/mixtral-8x7b-v2-0) (Mixtral 8x7b v2.0)

My journey with Mixtral started with me being unable to load the model in a single GPU. This results in an OOM errors:

<img src="../media_rs/OOM1.png" style="height:300px" />
<img src="../media_rs/OОM2.png" style="height:300px" />

Other techniques for optimization had bad reliability also. Using weight quantization with Bits and Bytes (bnb) was a hit or a miss, where 50% of the time the model failed to load with quantized weights, raising an error which said that bnb needs the "accelerate" library to be installed even though it is. In the times when I don't get dependency errors the gpu runs OOM again too even with weight quantization:

<img src="../media_rs/error1.png" style="height:300px" />

> Here it was even more weird, since I was using 2 T4 gpus but kaggle was showing I was utilizing the cpu even though it showed it had 30gb vram... It was a total mess. You can even see that under "Draft Session" it says that I indeed use 2x T4. 

In the end I used the offloading techniques by the mentioned duo Artyom Eliseev and Denis Mazur to load Mixtral 8x7b Instruct and test it. Here I encountered problems with the model's output, most probably because of my inability to set up proper inference. Although I manage to fix it later (more on that below), still it does not give the right answers (more precisely, the correct format).

He is an example of the model not working:

<img src="../media_rs/inf1.png" style="height:400px" />

> This can be seen in the provided Notebook 2 (Mixtral 8x7b v2.0)

Here is the code that generated the output. I first initialize the model and then compute a prompt (code works only in kaggle since the MMLU dataset is needed and the dependency for offloading). To see proof of work I suggest looking in the notebooks :)

#### Model initialization

In [ ]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"

config = AutoConfig.from_pretrained(quantized_model_name)

device = torch.device("cuda")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
offload_per_layer = 4
# offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
)

#### Inference

In [ ]:
def format_prompt(row):
    options = f"A: {row['A']} B: {row['B']} C: {row['C']} D: {row['D']}"
    prompt = f"<s> [INST] {row['prompt']} Options: {options} [/INST] Model answer: </s>"
    return prompt

def get_model_prediction(prompt):
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
    print("Generating prediction...\n")
    outputs = model.generate(input_ids, max_length=10)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def evaluate_predictions(ds):
    correct = 0
    for _, row in ds.iterrows():
        print("Formating...\n")
        formatted_prompt = format_prompt(row)
        print(f"Formated prompt: {formatted_prompt}\n")
        prediction = get_model_prediction(formatted_prompt)
        print(f"Prediction: {prediction}\n")
        
        if prediction.strip().upper() == row['answer'].strip().upper():
            correct += 1
        break
    accuracy = correct / len(ds)
    return accuracy

accuracy = evaluate_predictions(ds)
print(f"Model Accuracy: {accuracy*100:.2f}%")

After more research I made it work somewhat by using the format (having the input as a dict basically) shown in the quick tutorial in huggingface ([link](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1)). I then reused my old code and managed to get a little bit better results:

In [ ]:
def format_question(row):
    content = f"{row['prompt']} \nA: {row['A']} \nB: {row['B']} \nC: {row['C']} \nD: {row['D']}"
    return [{"role": "user", "content": content}]

def tokenize_input(formatted_input, tokenizer):
    inputs = tokenizer.apply_chat_template(formatted_input, return_tensors="pt").to("cuda")
    return inputs

def generate_answer(model, tokenized_input, tokenizer):
    outputs = model.generate(tokenized_input, max_new_tokens=10)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

Here I feed prompts to the model. I limit it to 3 because it still doesn't work perfectly, i.e. the model doesn't answer with just the letters of the answers.

In [ ]:
question_counter = 0 # since ds.iterrows() is not subscriptable
for index, row in ds.iterrows():
    formatted_input = format_question(row)
    tokenized_input = tokenize_input(formatted_input, tokenizer)
    answer = generate_answer(model, tokenized_input, tokenizer)
    print(answer)
    
    question_counter += 1
    
    if question_counter >= 3:
        break

Output:

<img src="../media_rs/chat1.png" style="height:400px" />

> Output can be seen in the kaggle notebook for proof of work.

Prompts are sentences that cut off at some point, because I have set `max_new_tokens=10`. For MMLU, the model must only answer with the letter of the correct answer, except if we change our protocol which can change the results also.

In conclusion, setting my inexperience aside, I believe that the researchers could have given more information on how they managed to get these test results. Although I am not saying that they haven't done their work properly, sharing such information is important for the future of LLMs and AI in general, since easier testing and reproduction by 3rd parties will be essential for early spotting of biases, mistakes and other kinds of unforeseen behavior by these algorithms, which if let loose while being unchecked, can cause a lot of trouble (to put it lightly).